In [1]:
# The purpose of these notebook is to create 
# The skeleton of a database, and populate it with HMP data
import sqlite3
import pandas as pd
#import numpy as np

In [2]:
def create_subject_entry(subject_id,
                         data,
                         idcolumn = 'MRN (Subject ID)',
                         gendercolumn = 'Host Gender'):
    """Create a pandas line for subject"""
    
    dat = data.loc[ data[idcolumn] == subject_id ]
    
    gender = dat[ gendercolumn ].unique()
    try:
        if gender.shape != (1,):
            raise ValueError
    except:
        print("Gender is not unique: {},{}".format(subject_id,gender))
        raise
    
    dat = pd.DataFrame({'subject_id' : subject_id,
                  'gender' : gender,
                  'internal_id' : subject_id,
                  'disease_state' : 'Healthy',
                  'population' : 'HMP',
                  'project' : 'HMP',
                 })
    
    return dat

In [3]:
# Filenames
sample_file = '/home/sur/micropopgen/data/HMP/hmp_catalogue_wgs_all_metadata.csv'
runs_file = '/home/sur/micropopgen/data/HMP/2017-07-17.all_runs/all_runs.txt'

In [4]:
# Read sample metadata
samples = pd.read_csv(sample_file)
samples.head()

# Read run metadata
runs = pd.read_csv(runs_file,sep='\t')
runs.head()

# Checking columns that have SRA sample ID (ie SRS###)
print(samples['Sequence Read Archive ID'].head())
print(runs['secondary_sample_accession'].head())

0    SRS011061
1    SRS011084
2    SRS011086
3    SRS011090
4    SRS011098
Name: Sequence Read Archive ID, dtype: object
0    SRS011061
1    SRS011061
2    SRS011084
3    SRS011084
4    SRS011084
Name: secondary_sample_accession, dtype: object


In [21]:
# Find samples that have duplicated rows and remove them
dups = samples['Sequence Read Archive ID'].value_counts()[ samples['Sequence Read Archive ID'].value_counts() > 1 ]
samples = samples[~(samples['Sequence Read Archive ID'].isin(dups.index) & samples['HMP Isolation Body Subsite'].isnull())]
dups = samples['Sequence Read Archive ID'].value_counts()[ samples['Sequence Read Archive ID'].value_counts() > 1 ]
dups

Series([], Name: Sequence Read Archive ID, dtype: int64)

In [22]:
# Dropping unnecessary columns from pandas data frames
samples_small = samples.drop(['NCBI Taxon ID', 'NCBI Project ID',
                              'Project Status', 'Gene Count',
                              'Reads Count', 'Size (KB)',
                              'GC Content', 'IMG/HMP ID',
                              'Sequencing Center',
                              'Sequencing Country',
                              'Funding Source', 'Sequencing Method',
                              'Assembly Methods','Gene Calling Methods',
                              'Addition Date',
                              'Last Modification Date',
                              'Isolation Source',
                              'HMP Body Product'],
                             axis = 1)
runs_small = runs.drop(['study_accession',
                        'secondary_study_accession',
                        'sample_accession',
                        'experiment_accession',
                        'study_title','study_alias',
                        'submission_accession','tax_id',
                        'scientific_name','instrument_platform',
                        'library_source','library_selection',
                        'read_count','base_count','center_name',
                        'first_public','last_updated',
                        'nominal_length','library_strategy',
                        'run_alias','fastq_bytes','fastq_md5',
                        'fastq_ftp','fastq_aspera',
                        'fastq_galaxy','submitted_bytes',
                        'submitted_md5','experiment_alias',
                        'submitted_ftp','submitted_aspera',
                        'submitted_galaxy','submitted_format',
                        'sra_bytes','sra_md5','sra_ftp',
                        'sra_aspera','sra_galaxy',
                        'cram_index_ftp','cram_index_aspera',
                        'cram_index_galaxy','sample_alias',
                        'broker_name','sample_title'],
                       axis = 1)
# print(samples_small.columns)
# print(samples_small.head())
# print(runs_small.columns)
# print(runs_small.head())

In [23]:
# Identify the subsites without body site
subsites = samples_small['HMP Isolation Body Subsite']
subsite_missing = subsites.isnull()
sites = samples_small.loc[subsite_missing,'HMP Isolation Body Site']
sites.value_counts(dropna=False)

gastrointestinal_tract    154
oral                        5
Name: HMP Isolation Body Site, dtype: int64

In [24]:
# Check body subsite for all GI samples
sites = samples_small['HMP Isolation Body Site']
sites.value_counts(dropna=False)
sites_gi = sites == 'gastrointestinal_tract'
subsite = samples_small.loc[ sites_gi, 'HMP Isolation Body Subsite']
subsite.value_counts(dropna=False)

NaN      154
Stool     97
Name: HMP Isolation Body Subsite, dtype: int64

In [25]:
# All samples from GI tract are from stool, so I will change the NaN to that
samples_small.loc[ samples_small['HMP Isolation Body Site'] == 'gastrointestinal_tract',
                  'HMP Isolation Body Subsite'] = 'Stool'
# Check body subsite for all GI samples
sites = samples_small['HMP Isolation Body Site']
sites.value_counts(dropna=False)
sites_gi = sites == 'gastrointestinal_tract'
subsite = samples_small.loc[ sites_gi, 'HMP Isolation Body Subsite']
subsite.value_counts(dropna=False)

Stool    251
Name: HMP Isolation Body Subsite, dtype: int64

In [26]:
# There are 5 oral samples that have no subsite. I will set them to 'oral'
samples_small.loc[ samples_small['HMP Isolation Body Subsite'].isnull() &
                  (samples_small['HMP Isolation Body Site'] == 'oral'),
                  'HMP Isolation Body Subsite'] = 'oral'

In [27]:
samples_small['HMP Isolation Body Site'].value_counts(dropna=False)

oral                      690
gastrointestinal_tract    251
airways                   138
urogenital_tract          128
skin                       55
Name: HMP Isolation Body Site, dtype: int64

In [28]:
samples_small['HMP Isolation Body Subsite'].value_counts(dropna=False)

Stool                           251
Tongue dorsum                   221
Supragingival plaque            210
Buccal mucosa                   185
Nares                           138
Posterior fornix                108
Retroauricular crease            54
Palatine tonsils                 20
Subgingival plaque               19
Throat                           13
Vaginal                          12
Vaginal introitus                 8
Attached Keratinized gingiva      8
Attached/Keratinized gingiva      6
oral                              5
Saliva                            2
Antecubital fossa                 1
Hard palate                       1
Name: HMP Isolation Body Subsite, dtype: int64

In [29]:
# Now I have homogenized the body subsites. I will fill the table
conn = sqlite3.connect('/home/sur/micropopgen/data/database/metagenomes.db')
c = conn.cursor()

In [30]:
# Iterate and add
for i,r in samples_small.iterrows():
    #print(r)
    values = (r['MRN (Subject ID)'],r['HMP Isolation Body Subsite'],
             r['Visit Number'],'NULL',
             r['Sequence Read Archive ID'],
             'Homo sapiens',
             r['Replicate Number'])
    #print(values)
    c.execute("insert into sample values (null,?,?,?,?,?,?,?)", values)
    print(r['Sequence Read Archive ID'])
conn.commit()
#conn.close()

SRS011061
SRS011084
SRS011086
SRS011090
SRS011098
SRS011105
SRS011111
SRS011115
SRS011126
SRS011132
SRS011134
SRS011140
SRS011144
SRS011152
SRS011239
SRS011243
SRS011247
SRS011255
SRS011263
SRS011269
SRS011271
SRS011302
SRS011306
SRS011310
SRS011343
SRS011355
SRS011397
SRS011405
SRS011452
SRS011529
SRS011584
SRS011586
SRS012273
SRS012279
SRS012281
SRS012285
SRS012291
SRS012294
SRS012663
SRS012849
SRS012902
SRS012916
SRS012918
SRS012922
SRS012924
SRS012934
SRS012969
SRS012983
SRS012985
SRS012989
SRS012998
SRS013098
SRS013155
SRS013158
SRS013164
SRS013170
SRS013215
SRS013216
SRS013234
SRS013239
SRS013252
SRS013258
SRS013261
SRS013269
SRS013476
SRS013502
SRS013506
SRS013521
SRS013533
SRS013542
SRS013637
SRS013638
SRS013639
SRS013687
SRS013705
SRS013711
SRS013723
SRS013740
SRS013800
SRS013818
SRS013825
SRS013836
SRS013876
SRS013879
SRS013881
SRS013942
SRS013945
SRS013946
SRS013947
SRS013948
SRS013949
SRS013950
SRS013951
SRS013956
SRS014107
SRS014124
SRS014126
SRS014235
SRS014271
SRS014287


In [31]:
#############
# Now we move to the runs table
# Merge tables to get subject ID
merged = pd.merge(samples_small, runs_small,
                  right_on='secondary_sample_accession',
                  left_on = 'Sequence Read Archive ID')
merged.head()
# runs_small.head()
# merged.columns
# print(merged['MRN (Subject ID)'].head())
# print(merged['subject_id'].head())
# print(merged.loc[ merged['subject_id'].notnull(), 'subject_id'].head())
# print(merged.loc[ merged['subject_id'].notnull(), 'MRN (Subject ID)'].head())

,Sequence Read Archive ID,HMP Isolation Body Site,HMP Isolation Body Subsite,Host Gender,MRN (Subject ID),Visit Number,Replicate Number,secondary_sample_accession,run_accession,instrument_model,library_name,library_layout,experiment_title,subject_id
0,SRS011061,gastrointestinal_tract,Stool,Female,158458797,1,0,SRS011061,SRR060370,Illumina Genome Analyzer II,Solexa-27630,PAIRED,Illumina Genome Analyzer II paired end sequenc...,NaN
1,SRS011061,gastrointestinal_tract,Stool,Female,158458797,1,0,SRS011061,SRR060371,Illumina Genome Analyzer II,Solexa-27630,PAIRED,Illumina Genome Analyzer II paired end sequenc...,NaN
2,SRS011084,gastrointestinal_tract,Stool,Male,158479027,1,0,SRS011084,SRR061903,Illumina Genome Analyzer II,Solexa-28680,PAIRED,Illumina Genome Analyzer II paired end sequenc...,NaN
3,SRS011084,gastrointestinal_tract,Stool,Male,158479027,1,0,SRS011084,SRR061904,Illumina Genome Analyzer II,Solexa-28680,PAIRED,Illumina Genome Analyzer II paired end sequenc...,NaN
4,SRS011084,gastrointestinal_tract,Stool,Male,158479027,1,0,SRS011084,SRR062102,Illumina Genome Analyzer II,Solexa-28680,PAIRED,Illumina Genome Analyzer II paired end sequenc...,NaN


In [32]:
conn = sqlite3.connect('/home/sur/micropopgen/data/database/metagenomes.db')
c = conn.cursor()
# Iterate and add
for i,r in merged.iterrows():
    #print(r)
    values = (r['secondary_sample_accession'],
              r['MRN (Subject ID)'],
              r['instrument_model'],
              r['library_layout'],
              r['run_accession'])
    #print(values)
    c.execute("insert into run values (null,?,?,?,?,?)", values)
    print(r['run_accession'])
conn.commit()
# conn.close()

SRR060370
SRR060371
SRR061903
SRR061904
SRR062102
SRR062103
SRR061957
SRR061958
SRR060388
SRR060389
SRR059326
SRR059327
SRR062051
SRR062052
SRR060398
SRR060399
SRR059320
SRR059321
SRR062045
SRR062046
SRR062096
SRR062097
SRR061987
SRR061988
SRR062068
SRR062069
SRR062013
SRR062014
SRR062055
SRR062056
SRR061907
SRR061908
SRR060442
SRR060443
SRR062053
SRR062054
SRR062031
SRR062032
SRR061911
SRR061912
SRR059854
SRR059855
SRR060416
SRR060417
SRR060400
SRR060401
SRR059918
SRR059919
SRR060438
SRR060439
SRR060412
SRR060413
SRR061168
SRR061170
SRR060410
SRR060411
SRR059928
SRR059929
SRR059324
SRR059325
SRR062049
SRR062050
SRR060420
SRR060421
SRR060392
SRR060393
SRR062072
SRR062073
SRR060402
SRR060403
SRR061164
SRR061166
SRR062426
SRR062428
SRR346668
SRR346703
SRR061153
SRR061173
SRR062434
SRR062483
SRR061144
SRR061145
SRR059412
SRR059413
SRR059414
SRR059415
SRR059448
SRR059449
SRR059450
SRR059451
SRR059410
SRR059411
SRR059452
SRR059453
SRR061901
SRR061902
SRR062104
SRR062105
SRR527872
SRR527921


SRR059403
SRR059874
SRR059875
SRR060372
SRR060373
SRR059400
SRR059401
SRR059354
SRR059355
SRR061923
SRR061924
SRR061925
SRR061926
SRR531987
SRR531997
SRR531998
SRR532491
SRR532494
SRR532496
SRR532502
SRR533932
SRR539795
SRR539798
SRR539826
SRR539828
SRR539987
SRR539988
SRR539990
SRR540004
SRR540419
SRR540435
SRR540442
SRR541344
SRR541350
SRR541352
SRR541360
SRR541365
SRR545467
SRR646441
SRR646445
SRR646451
SRR646453
SRR646456
SRR061973
SRR061974
SRR060376
SRR060377
SRR061985
SRR061986
SRR062066
SRR062067
SRR061971
SRR061972
SRR059466
SRR059467
SRR059464
SRR059465
SRR062011
SRR062012
SRR062043
SRR062044
SRR062084
SRR062085
SRR061979
SRR061980
SRR059394
SRR059395
SRR059396
SRR059397
SRR059398
SRR059399
SRR062058
SRR062059
SRR059912
SRR059913
SRR062001
SRR062002
SRR059916
SRR059917
SRR059906
SRR059907
SRR062041
SRR062042
SRR059872
SRR059873
SRR527924
SRR527961
SRR527995
SRR528040
SRR528075
SRR528120
SRR528170
SRR528276
SRR528295
SRR528327
SRR528412
SRR528450
SRR528503
SRR532395
SRR532404


SRR061240
SRR063494
SRR063495
SRR061396
SRR061400
SRR063747
SRR063789
SRR531256
SRR531490
SRR531494
SRR531495
SRR531671
SRR532332
SRR532349
SRR532531
SRR532549
SRR532551
SRR532552
SRR533292
SRR533297
SRR533312
SRR533313
SRR533742
SRR533749
SRR534439
SRR534571
SRR534574
SRR537797
SRR538093
SRR538232
SRR539573
SRR539575
SRR539590
SRR539597
SRR540683
SRR540693
SRR545264
SRR545274
SRR549436
SRR549440
SRR549612
SRR549618
SRR549619
SRR549628
SRR549630
SRR532294
SRR532301
SRR532325
SRR532837
SRR532865
SRR533689
SRR537831
SRR537959
SRR539537
SRR539545
SRR539550
SRR539556
SRR539566
SRR540472
SRR540476
SRR540868
SRR540871
SRR541138
SRR541141
SRR541154
SRR545492
SRR545497
SRR545498
SRR545508
SRR638771
SRR638842
SRR640445
SRR640464
SRR640528
SRR650080
SRR061361
SRR061415
SRR353625
SRR532688
SRR532689
SRR532692
SRR532698
SRR532699
SRR532702
SRR532703
SRR532704
SRR532707
SRR533478
SRR537753
SRR537909
SRR540555
SRR540556
SRR541048
SRR541053
SRR541063
SRR541069
SRR541074
SRR541372
SRR541384
SRR541393


SRR545247
SRR545256
SRR549757
SRR549781
SRR638784
SRR638806
SRR640334
SRR640348
SRR640397
SRR650082
SRR062328
SRR062329
SRR346698
SRR060039
SRR061573
SRR061589
SRR531378
SRR531385
SRR531386
SRR531497
SRR531527
SRR531578
SRR531580
SRR531583
SRR532326
SRR532428
SRR532436
SRR533775
SRR537962
SRR537980
SRR540458
SRR540460
SRR540471
SRR541119
SRR541122
SRR541134
SRR541149
SRR545258
SRR549763
SRR549769
SRR580766
SRR638862
SRR640350
SRR640354
SRR640504
SRR650045
SRR059856
SRR059857
SRR059818
SRR061399
SRR061417
SRR061559
SRR061561
SRR063708
SRR527917
SRR527976
SRR528008
SRR528103
SRR528143
SRR528195
SRR528271
SRR528312
SRR528340
SRR528405
SRR528435
SRR528478
SRR528532
SRR062325
SRR062373
SRR061927
SRR061928
SRR063514
SRR063525
SRR061409
SRR061418
SRR061277
SRR061288
SRR532297
SRR532308
SRR532315
SRR532322
SRR533196
SRR533202
SRR533218
SRR537979
SRR537996
SRR539471
SRR539484
SRR539486
SRR539493
SRR539497
SRR539500
SRR539505
SRR539510
SRR539519
SRR540880
SRR540889
SRR540893
SRR545486
SRR545504


SRR533562
SRR537805
SRR537809
SRR538238
SRR539432
SRR539443
SRR539447
SRR539454
SRR539873
SRR539875
SRR539888
SRR539893
SRR539902
SRR540591
SRR540601
SRR540607
SRR540608
SRR540609
SRR540616
SRR540619
SRR540716
SRR540721
SRR540732
SRR532156
SRR532167
SRR532171
SRR532179
SRR532389
SRR532394
SRR533138
SRR533832
SRR533833
SRR533834
SRR533847
SRR533953
SRR537705
SRR537732
SRR537836
SRR538079
SRR538141
SRR538189
SRR539856
SRR541003
SRR541013
SRR541023
SRR541032
SRR541092
SRR541100
SRR541112
SRR541113
SRR541249
SRR545598
SRR545608
SRR545613
SRR545628
SRR549713
SRR549721
SRR549728
SRR549731
SRR549740
SRR549742
SRR531973
SRR531976
SRR531977
SRR531993
SRR531999
SRR532001
SRR532505
SRR532959
SRR532984
SRR537904
SRR537928
SRR538117
SRR539807
SRR539994
SRR539998
SRR540000
SRR540008
SRR540410
SRR540417
SRR540436
SRR541307
SRR541321
SRR545537
SRR545553
SRR638800
SRR640394
SRR640426
SRR640429
SRR640507
SRR650071
SRR531965
SRR531984
SRR531990
SRR531995
SRR532527
SRR532529
SRR532967
SRR532981
SRR537794


SRR549664
SRR532358
SRR532359
SRR532366
SRR532631
SRR532634
SRR532638
SRR532646
SRR532652
SRR533352
SRR533362
SRR533805
SRR533806
SRR537687
SRR537730
SRR538078
SRR539679
SRR539687
SRR539692
SRR539694
SRR539697
SRR539705
SRR539707
SRR540644
SRR540658
SRR540668
SRR541158
SRR541180
SRR541184
SRR541187
SRR545636
SRR545641
SRR545651
SRR545668
SRR549638
SRR549646
SRR549647
SRR549661
SRR549670
SRR533167
SRR533169
SRR533452
SRR533458
SRR533469
SRR533483
SRR533484
SRR533717
SRR533879
SRR537838
SRR537971
SRR538240
SRR539251
SRR540525
SRR540541
SRR540548
SRR541382
SRR541399
SRR545315
SRR545319
SRR549489
SRR549492
SRR549506
SRR549517
SRR638811
SRR639510
SRR640396
SRR640472
SRR640550
SRR650046
SRR532839
SRR532844
SRR532851
SRR533673
SRR533680
SRR533686
SRR533772
SRR533779
SRR533793
SRR537958
SRR539535
SRR539562
SRR539945
SRR540469
SRR540482
SRR541257
SRR541264
SRR541271
SRR541281
SRR541288
SRR541290
SRR545241
SRR545243
SRR549759
SRR580776
SRR638767
SRR638854
SRR640399
SRR640427
SRR650092
SRR532455


In [33]:
###########
# Now we create the subject table
subjects = samples_small['MRN (Subject ID)'].unique()
subject_table = pd.DataFrame()
for s in subjects:
    t = create_subject_entry(s,data = samples_small)
    subject_table = subject_table.append(t)
subject_table.head()

,disease_state,gender,internal_id,population,project,subject_id
0,Healthy,Female,158458797,HMP,HMP,158458797
0,Healthy,Male,158479027,HMP,HMP,158479027
0,Healthy,Male,158499257,HMP,HMP,158499257
0,Healthy,Female,158742018,HMP,HMP,158742018
0,Healthy,Male,158802708,HMP,HMP,158802708


In [34]:
conn = sqlite3.connect('/home/sur/micropopgen/data/database/metagenomes.db')
c = conn.cursor()
# Iterate and add
for i,r in subject_table.iterrows():
    #print(r)
    values = (r['subject_id'],
              r['gender'],
              r['subject_id'],
              r['disease_state'],
              r['population'],
              r['project'])
    #print(values)
    c.execute("insert into subject values (?,?,?,?,?,?)", values)
    print(r['subject_id'])
conn.commit()
# conn.close()

158458797
158479027
158499257
158742018
158802708
158944319
158984779
159247771
159146620
159166850
159733294
158337416
158398106
158822939
158883629
159571453
159369152
159591683
159207311
159227541
159288231
159268001
763435843
763496533
763820215
763678604
763536994
763577454
763840445
763961826
700023342
763860675
764062976
763901136
763759525
764143897
763982056
764042746
764083206
764285508
764487809
764224817
764002286
764305738
764325968
764447348
764508039
764588959
764649650
764669880
765074482
159632143
159672603
159429842
159753524
159915365
160421117
159450072
159611913
159470302
159490532
159551223
159814214
159510762
160481808
160319967
160502038
160582958
160542498
160603188
158357646
160158126
160380657
160178356
160218816
160259276
765094712
765013792
765135172
765034022
765337473
765560005
763597684
765620695
764892411
160643649
160400887
160704339
160906640
160947100
160765029
700096416
161007791
161230322
158216035
158155345
158256496
638754422
533247696
809635352


In [35]:
conn.close()

In [36]:
#######
# Test that we can recall data
conn = sqlite3.connect('/home/sur/micropopgen/data/database/metagenomes.db')

In [37]:
#samples_small.head()
pd.read_sql_query("select * from sample limit 10;", conn)

,sample_id,subject_id,body_site,visit,age,SRS,host_species,replicate
0,1,158458797,Stool,1,NULL,SRS011061,Homo sapiens,0
1,2,158479027,Stool,1,NULL,SRS011084,Homo sapiens,0
2,3,158458797,Tongue dorsum,1,NULL,SRS011086,Homo sapiens,0
3,4,158458797,Buccal mucosa,1,NULL,SRS011090,Homo sapiens,0
4,5,158458797,Supragingival plaque,1,NULL,SRS011098,Homo sapiens,0
5,6,158458797,Nares,1,NULL,SRS011105,Homo sapiens,0
6,7,158458797,Posterior fornix,1,NULL,SRS011111,Homo sapiens,0
7,8,158479027,Tongue dorsum,1,NULL,SRS011115,Homo sapiens,0
8,9,158479027,Supragingival plaque,1,NULL,SRS011126,Homo sapiens,0
9,10,158479027,Nares,1,NULL,SRS011132,Homo sapiens,0


In [38]:
#samples_small.head()
pd.read_sql_query("select * from run limit 20;", conn)

,run_id,sample_id,subject_id,platform,library_strategy,SRR
0,1,SRS011061,158458797,Illumina Genome Analyzer II,PAIRED,SRR060370
1,2,SRS011061,158458797,Illumina Genome Analyzer II,PAIRED,SRR060371
2,3,SRS011084,158479027,Illumina Genome Analyzer II,PAIRED,SRR061903
3,4,SRS011084,158479027,Illumina Genome Analyzer II,PAIRED,SRR061904
4,5,SRS011084,158479027,Illumina Genome Analyzer II,PAIRED,SRR062102
5,6,SRS011084,158479027,Illumina Genome Analyzer II,PAIRED,SRR062103
6,7,SRS011086,158458797,Illumina Genome Analyzer II,PAIRED,SRR061957
7,8,SRS011086,158458797,Illumina Genome Analyzer II,PAIRED,SRR061958
8,9,SRS011090,158458797,Illumina Genome Analyzer II,PAIRED,SRR060388
9,10,SRS011090,158458797,Illumina Genome Analyzer II,PAIRED,SRR060389


In [39]:
#samples_small.head()
pd.read_sql_query("select * from subject limit 10;", conn)

,subject_id,gender,internal_id,disease_state,population,project
0,103092734,Male,103092734,Healthy,HMP,HMP
1,147406386,Male,147406386,Healthy,HMP,HMP
2,158114885,Female,158114885,Healthy,HMP,HMP
3,158155345,Male,158155345,Healthy,HMP,HMP
4,158216035,Male,158216035,Healthy,HMP,HMP
5,158256496,Female,158256496,Healthy,HMP,HMP
6,158337416,Female,158337416,Healthy,HMP,HMP
7,158357646,Male,158357646,Healthy,HMP,HMP
8,158398106,Male,158398106,Healthy,HMP,HMP
9,158458797,Female,158458797,Healthy,HMP,HMP


In [40]:
conn.close()